In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

Mounted at /content/drive


In [29]:
fire_data = pd.read_csv("/content/drive/MyDrive/Wildfire (1)/Datasets/merged_tp_precip_wind_fmc_all.csv")
fire_data.drop(["Unnamed: 0"], axis = 1, inplace = True)
fire_data.drop(["START_DATE"], axis = 1, inplace = True)
fire_data.drop(["END_DATE"], axis = 1, inplace = True)
def classify_ranges(df, column_name, ranges, classification_column):
    for range_name, range_values in ranges.items():
        df.loc[df[column_name].between(*range_values), classification_column] = range_name

# Example usage


classification_ranges = {
    'A': (0, 99),
    'B': (99.000000000001, 999),
    'C': (999.000000000001, 99999999999999)
}
  # Creating a new column to store the classifications
classify_ranges(fire_data, 'FIRE_SIZE', classification_ranges, 'FIRE_SIZE_CLASS')
fire_data.drop(["FIRE_SIZE"], axis = 1, inplace = True)
fire_data.groupby("FIRE_SIZE_CLASS").count()
fire_data.dropna(inplace = True)
fire_data['FIRE_SIZE_CLASS'] = fire_data['FIRE_SIZE_CLASS'].replace({'A':0 , 'B':1 , 'C':2 ,'D':3 , 'E':4 , 'F':5 , 'G':6 })
fire_data["DATE"] = fire_data["DATE"].astype(float)
x = fire_data.drop("FIRE_SIZE_CLASS", axis=1)
y = fire_data["FIRE_SIZE_CLASS"]
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = .3, random_state = 101)


In [32]:
fire_data.groupby("FIRE_SIZE_CLASS").count()

,LATITUDE,LONGITUDE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
FIRE_SIZE_CLASS,,,,,,,,,,
0,197739,197739,197739,197739,197739,197739,197739,197739,197739,197739
1,16518,16518,16518,16518,16518,16518,16518,16518,16518,16518
2,4063,4063,4063,4063,4063,4063,4063,4063,4063,4063


In [41]:
from collections import Counter
value_counts = Counter(Y_train)
for value, count in value_counts.items():
    print(f"Value {value}: Count {count}")

Value 0: Count 138507
Value 1: Count 11468
Value 2: Count 2849


In [30]:
X_train

,LATITUDE,LONGITUDE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
13345,36.433333,-112.099722,4005.0,20060628.0,252.0,107.0,169.0,76.0,2.155213,104.333
143376,35.771944,-106.748611,35043.0,20130708.0,223.0,105.0,157.0,25.0,0.731644,66.000
40998,37.333611,-107.301944,8007.0,20140726.0,327.0,79.0,178.0,0.0,2.032951,76.333
45407,39.759722,-107.562222,8045.0,20101003.0,152.0,53.0,92.0,0.0,1.989800,93.667
20590,36.632500,-113.555800,4015.0,20020602.0,311.0,189.0,250.0,0.0,3.890060,42.000
...,...,...,...,...,...,...,...,...,...,...
136767,39.852200,-115.019700,32033.0,20070810.0,201.0,86.0,142.0,0.0,3.934520,105.000
204614,48.890278,-121.623333,53073.0,20140816.0,234.0,160.0,184.0,0.0,2.054110,99.600
214353,42.973889,-109.674722,56035.0,20100901.0,88.0,26.0,64.0,25.0,2.046289,75.333
214539,43.234167,-109.941667,56035.0,20071002.0,54.0,-68.0,-9.0,25.0,1.287006,63.333


In [40]:
Y_train

13345     0
143376    0
40998     0
45407     0
20590     1
         ..
136767    0
204614    0
214353    0
214539    0
176991    0
Name: FIRE_SIZE_CLASS, Length: 152824, dtype: int64

In [28]:
X_train = tf.convert_to_tensor(X_train.values, dtype=tf.float32)
Y_train = tf.convert_to_tensor(Y_train.values, dtype=tf.int64)
X_test = tf.convert_to_tensor(X_test.values, dtype=tf.float32)
Y_test = tf.convert_to_tensor(Y_test.values, dtype=tf.int64)

AttributeError: ignored

In [26]:
print(X_train)
print(Y_train)
print(X_test)
print(Y_test)

tf.Tensor(
[[ 3.64333344e+01 -1.12099724e+02  4.00500000e+03 ...  7.60000000e+01
   2.15521312e+00  1.04333000e+02]
 [ 3.57719460e+01 -1.06748611e+02  3.50430000e+04 ...  2.50000000e+01
   7.31644034e-01  6.60000000e+01]
 [ 3.73336105e+01 -1.07301941e+02  8.00700000e+03 ...  0.00000000e+00
   2.03295064e+00  7.63330002e+01]
 ...
 [ 4.29738884e+01 -1.09674721e+02  5.60350000e+04 ...  2.50000000e+01
   2.04628921e+00  7.53330002e+01]
 [ 4.32341652e+01 -1.09941666e+02  5.60350000e+04 ...  2.50000000e+01
   1.28700650e+00  6.33330002e+01]
 [ 4.34628983e+01 -1.20586899e+02  4.10370000e+04 ...  0.00000000e+00
   2.28711295e+00  9.80000000e+01]], shape=(152824, 10), dtype=float32)
tf.Tensor([0 0 0 ... 0 0 0], shape=(152824,), dtype=int64)
tf.Tensor(
[[ 3.7222198e+01 -1.1243640e+02  4.9025000e+04 ...  2.5000000e+01
   2.0637066e+00  9.1500000e+01]
 [ 4.5453888e+01 -1.1092389e+02  3.0031000e+04 ...  0.0000000e+00
   1.9525589e+00  1.6000000e+01]
 [ 4.6851665e+01 -1.1172889e+02  3.0049000e+04 ..

In [25]:
class Softmax(tf.keras.Model):
    def __init__(self, n_inputs, n_outputs):
        super(Softmax, self).__init__()
        self.linear = tf.keras.layers.Dense(16, activation='relu', input_shape=(n_inputs,))
        self.linear1 = tf.keras.layers.Dense(32, activation='relu')
        self.linear2 = tf.keras.layers.Dense(16, activation='relu')
        self.linear3 = tf.keras.layers.Dense(n_outputs)

    def call(self, x):
        x = tf.nn.relu(self.linear(x))
        x = tf.nn.relu(self.linear1(x))
        x = tf.nn.relu(self.linear2(x))
        x = self.linear3(x)
        return tf.nn.log_softmax(x, axis=1)
model_softmax = Softmax(10, 3)
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
batch_size = 128
train_data = train_data.batch(batch_size)
train_iterator = iter(train_data)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
learning_rate = 0.0001
optimizer = tf.keras.optimizers.SGD(learning_rate)
Loss = []
epochs = 2

for epoch in range(epochs):
    running_loss = 0
    print(epoch)
    train_iterator = iter(train_data)

    for _ in range(len(X_train) // batch_size):
        x, y = train_iterator.get_next()

        with tf.GradientTape() as tape:
            y_pred = model_softmax(x)
            loss = loss_fn(y, y_pred)

        Loss.append(loss.numpy())
        gradients = tape.gradient(loss, model_softmax.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_softmax.trainable_variables))

        running_loss += loss.numpy()

logits = model_softmax(X_test)
y_pred = tf.argmax(logits, axis=1)

Streaming output truncated to the last 5000 lines.
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 16), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 32), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 16), dtype=float32)
tf.Tensor(
[[ 4.61877785e+01 -1.13654724e+02  3.00390000e+04 ...  0.00000000e+00
   5.93967080e-01  6.00000000e+01]
 [ 4.24900017e+01 -1.11050003e+02  1.60070000e+04 ...  2.50000000e+01
   2.69283938e+00  1.12000000e+02]
 [ 4.42730560e+01 -1.24051941e+02  4.10390000e+04 ...  0.00000000e+00
   3.33779502e+00  5.10000000e+01]
 ...
 [ 4.25663986e+01 -1.22841400e+02  4.10290000e+04 ... 

KeyboardInterrupt: ignored

In [20]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95     59232
           1       0.00      0.00      0.00      5050
           2       0.00      0.00      0.00      1214

    accuracy                           0.90     65496
   macro avg       0.30      0.33      0.32     65496
weighted avg       0.82      0.90      0.86     65496



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
print(accuracy_score(Y_test, y_pred))

0.9043605716379626


In [22]:
print(confusion_matrix(Y_test, y_pred))

[[59232     0     0]
 [ 5050     0     0]
 [ 1214     0     0]]
